In [ ]:
# # 압축 풀기
# import shutil

# filename = 'data/NIKL_NEWSPAPER_2020_v1.1.zip'
# extract_dir = './data/NIKL_NEWSPAPER_2020_v1.1'
# archive_format = "zip"

# shutil.unpack_archive(filename, extract_dir, archive_format)

## 신문 말뭉치

In [ ]:
# !git clone https://github.com/ko-nlp/Korpora
# !python setup.py install

In [ ]:
# !pip install pandas==1.1.5
# !pip install Korpora
# !pip install ray
# !pip install kss
# !python -m pip install regex

In [ ]:
import pandas as pd
from tqdm import tqdm
import kss
import re

import ray
from itertools import chain

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
from Korpora import Korpora

news_path = "./data/NIKL_NEWSPAPER_2020_v1.1/newspaper"
modu_news = Korpora.load("modu_news", root_dir=news_path)

In [ ]:
len(modu_news.get_all_texts())

In [ ]:
splited_sentences = []
corpus = modu_news.get_all_texts()

In [ ]:
corpus = corpus[0:20000]

In [ ]:
ray.init()

In [ ]:
@ray.remote
def split_sentences(corpus):
    splited_sentences = []
    
    for sentences in tqdm(corpus):
        if '_ ' in sentences: # 시상식 소감. 구어체라 사용 안함
            continue
            
        title_idx = sentences.find('\n')
        sentences = sentences[title_idx+1:] # 제목 제거

        sentences = re.sub(r'\s', ' ', sentences)
        sentences = re.sub('\u200b', ' ', sentences)
        sentences = re.sub('[◇◈◆▲△▷▶■ㅇ♡❤♥★�]','',sentences)
        sentences = sentences.replace("\"", "")
        
        kss_sentence = kss.split_sentences(sentences, backend='mecab', num_workers=4)
        splited_sentences.extend(kss_sentence)
                        
    return splited_sentences

In [ ]:
def chunker_list(seq, size):
    return (seq[i::size] for i in range(size))

# 프로세스 개수
process_num = 10

# 프로세스 개수만큼 corpus 분리
sentences_chunk = list(chunker_list(corpus, process_num))

# ray를 이용해 멀티프로세싱
futures = [split_sentences.remote(sentences_chunk[i]) for i in range(process_num)]

# 결과를 1-d로 합치기
results = ray.get(futures)
results = list(chain.from_iterable(results))

In [ ]:
len(results)

In [ ]:
no_spacing = []

for s in results:
    no_spacing.append(s.replace(" ",""))

In [ ]:
import random

random_spacing = []

for sentence in tqdm(no_spacing):
    k = len(sentence)//3

    tmp = sorted(random.sample(range(1,len(sentence)),k))
    
    tmp_list = list(sentence)
    for i, val in enumerate(tmp):
        tmp_list.insert(val+i, ' ')
    random_spacing.append(''.join(tmp_list))

In [ ]:
data = {'correct_sentence':results, 'wrong_sentence':random_spacing}
df = pd.DataFrame(data = data, columns=['correct_sentence','wrong_sentence'])

In [ ]:
df.to_csv("./news_data.csv",sep=',')

In [ ]:
len(df)

In [ ]:
ray.shutdown()

### 문어 말뭉치

In [ ]:
import json
import pandas as pd
import os

wr_path = "./data/NIKL_WRITTEN(v1.0)/written"
file_list = os.listdir(wr_path)
# print(file_list)
# print ("file_list: {}".format(file_list))

In [ ]:
wr_data = []
category = {}
annotation = {}
sentence = []

for f in file_list:
    if f[0] == ".":
        continue
        
    f = "./data/NIKL_WRITTEN(v1.0)/written/" + f
    wr_data = json.load(open(f))
    
    paragraph = wr_data['document'][0]['paragraph']

    ca = wr_data['metadata']['category']
    # an = wr_data['metadata']['annotation_level'][0]
    
    # 문어 말뭉치 카테고리 확인
    if ca in category:
        category[ca] += 1
    else:
        category[ca] = 1

    for p in range(len(paragraph)):
        sentence.append(paragraph[p]['form'])
    
print(len(sentence))